In [1]:
import pandas as pd
import os
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import numpy as np
# Import supplementary visualization code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline

 


### Setting files and directories

In [5]:
path_to_data= "/home/ubuntu/udacity/CodeGladiator/invesco/data"

transaction_file = "Code-Gladiators-Transaction.csv"
#investment_exp_file = "Code-Gladiators-InvestmentExperience.csv"
investment_exp_file = "imputed_investment_exp.csv"
investment_segment = "investment_vehicle_segment.csv"
aum_file = "Code-Gladiators-AUM.csv"
activity_file = "Code-Gladiators-Activity.csv"

test_file = "test_data.csv"



### reading csv files into pandas dataframe

In [6]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))

In [7]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score, accuracy_score
# time.time() return time in seconds since the Epoch
from time import time

beta = 0.5

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    X_train = X_train[:sample_size]
    y_train = y_train[:sample_size]
    
    start = time() # Get start time
    learner.fit(X_train, y_train)
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train)
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end-start
            
    # TODO: Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(y_train, predictions_train)
        
    # TODO: Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples
    results['f_train'] = fbeta_score(y_train, predictions_train, beta=beta)
        
   # TODO: Compute F-score on the test set
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=beta)
       
    # Success
    print ("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [8]:
#column_list= ['AUM_investor_log','Counts_investor','Shares_investor_log','AUM_advisor_log','Shares_advisor_log','Rating','1 Yr % Rank','3 Yr % Rank','1 Yr Return','3 Yr Return','Net Flows','Morningstar_Category_Rating','Transaction_Type']


column_list= [
    'Transaction_Type',
#    'Shares_investor',
#    'AUM_investor',
     'Counts_investor',
#     'Counts_advisor',
#	'Shares_advisor',
#	'AUM_advisor',
#	'Morningstar Category',
#	'Investment',
	'Rating',
	'1 Yr % Rank',
	'3 Yr % Rank',
#	'5 Yr % Rank',
#	'10 Yr % Rank',
	'1 Yr Return',
	'3 Yr Return',
#	'5 Yr Return',
#	'10 Yr Return',
#	'1 Yr Excess Return vs Primary Ix',
#	'3 Yr Excess Return vs Primary Ix',
#	'5 Yr Excess Return vs Primary Ix',
#	'10 Yr Excess Return vs Primary Ix',
#	'1 Yr Excess Return vs Category Ix',
#	'3 Yr Excess Return vs Category Ix',
#	'5 Yr Excess Return vs Category Ix',
#	'10 Yr Excess Return vs Category Ix',
	'Net Flows',
	'Morningstar_Category_Rating',
#	'investment_vehicle_segment',
	'AUM_investor_log',
	'Shares_investor_log',
	'AUM_advisor_log',
	'Shares_advisor_log',
	]


### processing data

Grouping data by unique advisor id and month. Final output will store sums of the assets under managements and shares for each advisor in particular month

In [13]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))




grouped_advisor_aum_df = aum_df.groupby(['Unique_Advisor_Id','Month']).agg({'AUM': 'sum','Shares':'sum','Unique_Investment_Id':'count'}).reset_index().rename(columns={'Unique_Investment_Id':'Counts'})
grouped_investment_aum_df = aum_df.groupby(['Unique_Investment_Id','Month']).agg({'AUM': 'sum','Shares':'sum','Unique_Advisor_Id':'count'}).reset_index().rename(columns={'Unique_Advisor_Id':'Counts'})

grouped_investment_aum_df['Year'],grouped_investment_aum_df['Month']=grouped_investment_aum_df['Month'].str.split(' /', 1).str
grouped_advisor_aum_df['Year'],grouped_advisor_aum_df['Month']=grouped_advisor_aum_df['Month'].str.split(' /', 1).str

transaction_df['Year'],transaction_df['Month']=transaction_df['Month'].str.split(' /', 1).str
investment_exp_df['Year'],investment_exp_df['Month']=investment_exp_df['Month'].str.split(' /', 1).str
aum_df['Year'],aum_df['Month']=aum_df['Month'].str.split(' /', 1).str
activity_df['Year'],activity_df['Month']=activity_df['Month'].str.split(' /', 1).str

grouped_investment_aum_df['Mapping_Month']= grouped_investment_aum_df['Month'].astype(int).apply(lambda x : x+1)
grouped_advisor_aum_df['Mapping_Month']= grouped_advisor_aum_df['Month'].astype(int).apply(lambda x : x+1)
investment_exp_df['Mapping_Month']= investment_exp_df['Month'].astype(int).apply(lambda x : x+1)
aum_df['Mapping_Month']= aum_df['Month'].astype(int).apply(lambda x : x+1)
activity_df['Mapping_Month']= activity_df['Month'].astype(int).apply(lambda x : x+1)

transaction_df['Month'] =  transaction_df['Month'].astype(int).apply(lambda x : x+0)

final_transaction = pd.merge(transaction_df, grouped_investment_aum_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_investor','Year_x' : 'Year', 'Counts' : 'Counts_investor', 'Shares': 'Shares_investor','Month_y':'Month_actual'})
final_transaction = final_transaction.drop('Year_y', 1)

test_transaction = pd.merge(test_df, grouped_investment_aum_df[grouped_investment_aum_df["Mapping_Month"]== 13], on="Unique_Investment_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_investor','Counts' : 'Counts_investor', 'Shares': 'Shares_investor'})

final_transaction = pd.merge(final_transaction, grouped_advisor_aum_df, left_on=["Month","Unique_Advisor_Id"],right_on=["Mapping_Month","Unique_Advisor_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Mapping_Month_x': 'Mapping_Month'})
final_transaction = final_transaction.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction = pd.merge(test_transaction, grouped_advisor_aum_df[grouped_advisor_aum_df["Mapping_Month"]== 13], on="Unique_Advisor_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_advisor','Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Month_x':'Month','Mapping_Month_x':'Mapping_Month'})
test_transaction = test_transaction.drop(['Year_x','Year_y','Mapping_Month_y','Month_y','Mapping_Month'], 1)

investment_exp_df['investment_vehicle_segment']= investment_segment_df['investment_vehicle_segment']
investment_exp_df= investment_exp_df[investment_exp_df['Year']=='2016']

final_transaction_with_exp = pd.merge(final_transaction, investment_exp_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction_with_exp = final_transaction_with_exp.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Mapping_Month_x':'Mapping_Month'})
final_transaction_with_exp = final_transaction_with_exp.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction_with_exp = pd.merge(test_transaction, investment_exp_df[investment_exp_df["Mapping_Month"]== 13 ], on="Unique_Investment_Id", how="left")

'''
final_transaction_with_exp['AUM_investor_log'] = np.log(final_transaction_with_exp['AUM_investor'])
final_transaction_with_exp['Shares_investor_log'] = np.log(final_transaction_with_exp['Shares_investor'])

final_transaction_with_exp['AUM_advisor_log'] = np.log(final_transaction_with_exp['AUM_advisor'])
final_transaction_with_exp['Shares_advisor_log'] = np.log(final_transaction_with_exp['Shares_advisor'])


test_transaction_with_exp['AUM_investor_log'] = np.log(test_transaction_with_exp['AUM_investor'])
test_transaction_with_exp['Shares_investor_log'] = np.log(test_transaction_with_exp['Shares_investor'])

test_transaction_with_exp['AUM_advisor_log'] = np.log(test_transaction_with_exp['AUM_advisor'])
test_transaction_with_exp['Shares_advisor_log'] = np.log(test_transaction_with_exp['Shares_advisor'])
'''
final_transaction_with_exp['AUM_investor_log'] = np.log(final_transaction_with_exp['AUM_investor']/final_transaction_with_exp['Counts_investor'])
final_transaction_with_exp['Shares_investor_log'] = np.log(final_transaction_with_exp['Shares_investor']/final_transaction_with_exp['Counts_investor'])

final_transaction_with_exp['AUM_advisor_log'] = np.log(final_transaction_with_exp['AUM_advisor']/final_transaction_with_exp['Counts_advisor'])
final_transaction_with_exp['Shares_advisor_log'] = np.log(final_transaction_with_exp['Shares_advisor']/final_transaction_with_exp['Counts_advisor'])


test_transaction_with_exp['AUM_investor_log'] = np.log(test_transaction_with_exp['AUM_investor']/test_transaction_with_exp['Counts_investor'])
test_transaction_with_exp['Shares_investor_log'] = np.log(test_transaction_with_exp['Shares_investor']/test_transaction_with_exp['Counts_investor'])

test_transaction_with_exp['AUM_advisor_log'] = np.log(test_transaction_with_exp['AUM_advisor']/test_transaction_with_exp['Counts_advisor'])
test_transaction_with_exp['Shares_advisor_log'] = np.log(test_transaction_with_exp['Shares_advisor']/test_transaction_with_exp['Counts_advisor'])
final_transaction_with_exp= final_transaction_with_exp.dropna()


required_train_df = final_transaction_with_exp.filter(column_list)

required_train_df['Transaction_Type']= required_train_df.apply(lambda x: 0 if x['Transaction_Type']== 'P' else 1, axis=1)

required_test_df = test_transaction_with_exp.filter(column_list)


#required_test_df['Rating'] = required_test_df['Rating'].astype(float)

required_test_df=required_test_df.fillna(required_test_df.median())

# Split the data into features and target label
transaction_type = required_train_df['Transaction_Type']
features_raw = required_train_df.drop('Transaction_Type', axis = 1)
test_raw = required_test_df


from sklearn.preprocessing import LabelEncoder
var_mod = column_list
#var_mod.remove('Transaction_Type')
le = LabelEncoder()
for i in var_mod:
    if i == 'Transaction_Type':
        continue
    features_raw[i] = le.fit_transform(features_raw[i])
    test_raw[i] = le.fit_transform(test_raw[i])

from sklearn.cross_validation import train_test_split

# Split the 'features' and 'transaction_type' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_raw, transaction_type, test_size = 0.2, random_state = 0)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

#############################################################################


# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# TODO: Initialize the three models
clf_A = LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)
clf_B = LinearSVC(random_state=101)
clf_C = GaussianNB()
clf_Ada = AdaBoostClassifier()
clf_Grad = GradientBoostingClassifier()
clf_KNN = KNeighborsClassifier()
clf_Dec = DecisionTreeClassifier()
clf_SGD = SGDClassifier()


# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
n_train = len(y_train)
samples_1 = int(n_train * 0.01)
samples_10 = int(n_train * 0.1)
samples_100 = n_train

# Collect results on the learners
results = {}
#for clf in [clf_A, clf_B, clf_C, clf_Ada, clf_Grad,clf_KNN ,clf_Dec, clf_SGD]:
for clf in [clf_Grad]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)
        
display(results)


from sklearn.externals import joblib

best_clf= clf_A
filename = 'logistic_regression_best_invesco.joblib.pkl'

_ = joblib.dump(best_clf, filename, compress=9)

clf_loaded = joblib.load(filename)


pred = clf_loaded.predict(test_raw)
pred_prob = clf_loaded.predict_proba(test_raw)

pred_prob = pd.DataFrame(pred_prob[:,1],columns=["Propensity_Score"])
display(pred_prob[:5])

pred_df= pd.DataFrame(pred,columns=["Redeem_Status"])

pred_df=pred_df.replace([0,1],['NO','YES'])
pred_df.head()

pred_df['Redeem_Status'].value_counts()





/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:69: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:71: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:72: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:76: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:79: RuntimeWarning: divide by zero encountered in log


Training set has 100161 samples.
Testing set has 25041 samples.


'\n#############################################################################\n\n\n# TODO: Import the three supervised learning models from sklearn\nfrom sklearn.linear_model import LogisticRegression, SGDClassifier\nfrom sklearn.svm import LinearSVC, SVC\nfrom sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.naive_bayes import GaussianNB\nfrom sklearn.tree import DecisionTreeClassifier\n\n# TODO: Initialize the three models\nclf_A = LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class=\'ovr\', n_jobs=1,\n          penalty=\'l2\', random_state=None, solver=\'newton-cg\', tol=0.0001,\n          verbose=0, warm_start=False)\nclf_B = LinearSVC(random_state=101)\nclf_C = GaussianNB()\nclf_Ada = AdaBoostClassifier()\nclf_Grad = GradientBoostingClassifier()\nclf_KNN = KNeighborsClassifier()\nclf_Dec = DecisionTreeClassif

In [14]:
X = X_train.as_matrix()
y= y_train.as_matrix()
X_submission = X_test.as_matrix()
y_check_test = y_test.as_matrix()

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
np.random.seed(0)  # seed to shuffle the train set

n_folds = 10
verbose = True
shuffle = False


if shuffle:
    idx = np.random.permutation(y.size)
    X = X[idx]
    y = y[idx]

skf = list(StratifiedKFold(y, n_folds))

clfs = [RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        LogisticRegression(),
        DecisionTreeClassifier(),
#        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        KNeighborsClassifier(),
        GaussianNB(),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

print ("Creating train and test sets for blending.")

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print("Fold", i)
        
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:, 1]
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)


print ("Blending.")
clf = LogisticRegression()
clf.fit(dataset_blend_train, y)
#y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
y_submission = clf.predict(dataset_blend_test)


print ("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_check_test, y_submission)))
print ("F-score on testing data: {:.4f}".format(fbeta_score(y_check_test, y_submission, beta = 1)))

Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e

In [15]:
X = features_raw.as_matrix()
y= transaction_type.as_matrix()
X_submission = test_raw.as_matrix()
#y_check_test = y_test.as_matrix()

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
np.random.seed(0)  # seed to shuffle the train set

n_folds = 10
verbose = True
shuffle = False


if shuffle:
    idx = np.random.permutation(y.size)
    X = X[idx]
    y = y[idx]

skf = list(StratifiedKFold(y, n_folds))

clfs = [RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        LogisticRegression(),
        DecisionTreeClassifier(),
#        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        KNeighborsClassifier(),
        GaussianNB(),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

print ("Creating train and test sets for blending.")

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print("Fold", i)
        
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:, 1]
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)


print ("Blending.")
clf = LogisticRegression()
clf.fit(dataset_blend_train, y)
y_submission_prob = clf.predict_proba(dataset_blend_test)[:, 1]
y_submission = clf.predict(dataset_blend_test)

print('done')

#print ("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_check_test, y_submission)))
#print ("F-score on testing data: {:.4f}".format(fbeta_score(y_check_test, y_submission, beta = 1)))

Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e

In [17]:
y_submission_prob = clf.predict_proba(dataset_blend_test)
y_submission = clf.predict(dataset_blend_test)

pred_prob= y_submission_prob
pred = y_submission
#pred = clf_loaded.predict(test_raw)
#pred_prob = clf_loaded.predict_proba(test_raw)

pred_prob = pd.DataFrame(pred_prob[:,1],columns=["Propensity_Score"])
display(pred_prob[:5])

pred_df= pd.DataFrame(pred,columns=["Redeem_Status"])

pred_df=pred_df.replace([0,1],['NO','YES'])
pred_df.head()

pred_df['Redeem_Status'].value_counts()

,Propensity_Score
0,0.449298
1,0.580018
2,0.423865
3,0.569235
4,0.607528


NO     6034
YES    2680
Name: Redeem_Status, dtype: int64

In [12]:
result = pd.concat([test_df, pred_prob, pred_df], axis=1)

result.head()


,Unique_Advisor_Id,Unique_Investment_Id,Propensity_Score,Redeem_Status
0,1000103,14147,0.786417,YES
1,1000103,3534,0.415081,NO
2,1000103,3651,0.169032,NO
3,1000103,7668,0.387780,NO
4,1000103,9339,0.508985,YES


In [13]:
result.to_csv('test_data_v3.csv',index=False)